## Changes made to other packages:
- In shap/explainers/_deep/deep_tf.py:
    remove leading underscore in _record_gradient (due to change in tensorflow)
- In shap/explainers/tf_utils.py:
    Explicitly add support for TensorflowRep type in functions to get inputs and outputs:
       elif str(type(model)).endswith("TensorflowRep'>"):
           return model.tensor_dict[model.outputs[0]]
    and the same for inputs but with model.inputs
- In onnx-tf/backend_rep.py:
    Add a __call__ method to TensorflowRep that runs self.run, so the model can be executed by calling it (as is done by shap):
        def __call__(self, *args, **kwargs):
            return self.run(*args, **kwargs)
            
These changes have been applied to the shap and onnx-tensorflow repositories at  
https://github.com/dianna-ai/shap  
https://github.com/dianna-ai/onnx-tensorflow

In [31]:
from pathlib import Path
import onnx
from onnx_tf.backend import prepare
import numpy as np
import tensorflow as tf
import shap

In [32]:
model = onnx.load('/Users/loostrum/surfdrive/Shared/datasets/mnist/mnist_model.onnx')

In [34]:
tf_rep = prepare(model, gen_tensor_dict=True)
tf_rep.tensor_dict

{'input': <tf.Tensor 'input:0' shape=(None, 1, 28, 28) dtype=float32>,
 'layer1.0.weight': <tf.Variable 'layer1.0.weight:0' shape=(16, 1, 5, 5) dtype=float32>,
 'layer1.0.bias': <tf.Variable 'layer1.0.bias:0' shape=(16,) dtype=float32>,
 'layer2.0.weight': <tf.Variable 'layer2.0.weight:0' shape=(32, 16, 5, 5) dtype=float32>,
 'layer2.0.bias': <tf.Variable 'layer2.0.bias:0' shape=(32,) dtype=float32>,
 'fc1.weight': <tf.Variable 'fc1.weight:0' shape=(32, 1568) dtype=float32>,
 'fc1.bias': <tf.Variable 'fc1.bias:0' shape=(32,) dtype=float32>,
 'fc2.weight': <tf.Variable 'fc2.weight:0' shape=(2, 32) dtype=float32>,
 'fc2.bias': <tf.Variable 'fc2.bias:0' shape=(2,) dtype=float32>,
 '26': <tf.Variable '26:0' shape=(1,) dtype=int64>,
 '_onnx_tf_internal_is_training': <tf.Tensor 'PlaceholderWithDefault:0' shape=() dtype=bool>,
 '9': <tf.Tensor 'transpose_2:0' shape=(None, 16, 28, 28) dtype=float32>,
 '10': <tf.Tensor 'transpose_4:0' shape=(None, 16, 14, 14) dtype=float32>,
 '11': <tf.Tensor '

In [21]:
tf_rep.export_graph('tf_proto')

INFO:tensorflow:Assets written to: tf_proto/assets


INFO:tensorflow:Assets written to: tf_proto/assets


In [22]:
tf_model = tf.saved_model.load('tf_proto')

In [35]:
data = np.random.random((1, 1, 28, 28)).astype(np.float32)
exp = shap.DeepExplainer(tf_rep, data)

In [36]:
exp.shap_values(data)

TypeError: in user code:

    /Users/loostrum/projects/DIANNA/converters/onnx_to_tf/shap/shap/explainers/_deep/deep_tf.py:240 grad_graph  *
        out = self.model(shap_rAnD)
    /Users/loostrum/projects/DIANNA/converters/onnx_to_tf/onnx-tensorflow/onnx_tf/backend_rep.py:70 __call__  *
        return self.run(*args, **kwargs)
    /Users/loostrum/projects/DIANNA/converters/onnx_to_tf/onnx-tensorflow/onnx_tf/backend_rep.py:108 run  *
        input_dict[k] = tf.constant(v)
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py:271 constant  **
        return _constant_impl(value, dtype, shape, name, verify_shape=False,
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py:288 _constant_impl
        tensor_util.make_tensor_proto(
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/framework/tensor_util.py:457 ma

## Shap is not working with our converted model, let's try with a native Keras model

In [25]:
model = tf.keras.applications.MobileNetV2()

In [26]:
model.inputs

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>]

In [27]:
model.outputs

[<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>]

In [28]:
type(model)

keras.engine.functional.Functional

In [29]:
data = np.random.random(size=(1, 224, 224, 3))
explainer = shap.DeepExplainer(model, data=data)

In [30]:
explainer.shap_values(data)

StagingError: in user code:

    /Users/loostrum/projects/DIANNA/converters/onnx_to_tf/shap/shap/explainers/_deep/deep_tf.py:245 grad_graph  *
        x_grad = tape.gradient(out, shap_rAnD)
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/eager/backprop.py:1084 gradient  **
        flat_grad = imperative_grad.imperative_grad(
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/eager/imperative_grad.py:71 imperative_grad
        return pywrap_tfe.TFE_Py_TapeGradient(
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/eager/backprop.py:148 _gradient_function
        grad_fn = ops._gradient_registry.lookup(op_name)  # pylint: disable=protected-access
    /Users/loostrum/pyenv/versions/3.9.1/envs/onnx/lib/python3.9/site-packages/tensorflow/python/framework/registry.py:99 lookup
        raise LookupError(

    LookupError: gradient registry has no entry 

There is an open issue about a LookupError like this: https://github.com/slundberg/shap/issues/1406